In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/new-york-times-hardcover-fiction-best-sellers/nyt2.json


In [2]:
!pip install pyspark

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 310.8/310.8 MB 3.3 MB/s eta 0:00:0000:0100:01
  Preparing metadata (setup.py) ... done
  Created wheel for pyspark: filename=pyspark-3.4.0-py2.py3-none-any.whl size=311317145 sha256=4c6324abd80155029899bcdae88edeaa4b38f60e440f47aa811e405ee6f75322
  Stored in directory: /root/.cache/pip/wheels/81/1b/c2/e41f4bddafe5564d11b2414a62dd7f5d75fcf65b2d7b7805c4
Successfully built pyspark


In [3]:
import pandas as pd
import numpy as np 
from datetime import date, timedelta, datetime 
import time 
import pyspark
from pyspark.sql import SparkSession , SQLContext
from pyspark.context import SparkContext
from pyspark.sql.functions import *
from pyspark.sql.types import *

# Initialize Spark Session 
- Initialize Spark Session. 
- Dataframe can be created and registered as tables
- SQL tables can be executed,cached,and parquet/csv/avro/json formatted data can be read

In [4]:
sc = SparkSession.builder.appName('PySparkSqlTutorial').getOrCreate()

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
23/04/28 06:19:48 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
/opt/conda/lib/python3.7/site-packages/pyspark/context.py:317: FutureWarning: Python 3.7 support is deprecated in Spark 3.4.
  warnings.warn("Python 3.7 support is deprecated in Spark 3.4.", FutureWarning)


sc = SparkSession.builder.appName("PysparkExample").config ("spark.sql.shuffle.partitions", "50").config("spark.driver.maxResultSize","5g").config


In [5]:
sc

In [6]:
# Read dataframe 
df = sc.read.json("/kaggle/input/new-york-times-hardcover-fiction-best-sellers/nyt2.json")
#TXT files
# dataframe_txt = sc.read.text('./data/text_data.txt')

#CSV files
# dataframe_csv = sc.read.csv('./data/csv_data.csv')

#PARQUET files
# dataframe_parquet = sc.read.load('./data/parquet_data.parquet')

In [7]:
# Lets look at the data
df.show(5)

+--------------------+--------------------+-----------------+-----------------+--------------------+-------------+-----------------+-------------+----+--------------+--------------------+-------------+
|                 _id|  amazon_product_url|           author| bestsellers_date|         description|        price|   published_date|    publisher|rank|rank_last_week|               title|weeks_on_list|
+--------------------+--------------------+-----------------+-----------------+--------------------+-------------+-----------------+-------------+----+--------------+--------------------+-------------+
|{5b4aa4ead3089013...|http://www.amazon...|    Dean R Koontz|{{1211587200000}}|Odd Thomas, who c...|   {null, 27}|{{1212883200000}}|       Bantam| {1}|           {0}|           ODD HOURS|          {1}|
|{5b4aa4ead3089013...|http://www.amazon...|  Stephenie Meyer|{{1211587200000}}|Aliens have taken...|{25.99, null}|{{1212883200000}}|Little, Brown| {2}|           {1}|            THE HOST|     

In [8]:
# Fetch the column name 
df.columns

['_id',
 'amazon_product_url',
 'author',
 'bestsellers_date',
 'description',
 'price',
 'published_date',
 'publisher',
 'rank',
 'rank_last_week',
 'title',
 'weeks_on_list']

# Simple data inspection 

In [9]:
# Data types
df.dtypes

[('_id', 'struct<$oid:string>'),
 ('amazon_product_url', 'string'),
 ('author', 'string'),
 ('bestsellers_date', 'struct<$date:struct<$numberLong:string>>'),
 ('description', 'string'),
 ('price', 'struct<$numberDouble:string,$numberInt:string>'),
 ('published_date', 'struct<$date:struct<$numberLong:string>>'),
 ('publisher', 'string'),
 ('rank', 'struct<$numberInt:string>'),
 ('rank_last_week', 'struct<$numberInt:string>'),
 ('title', 'string'),
 ('weeks_on_list', 'struct<$numberInt:string>')]

In [10]:
#display content of dataframe
df.show(2)

+--------------------+--------------------+---------------+-----------------+--------------------+-------------+-----------------+-------------+----+--------------+---------+-------------+
|                 _id|  amazon_product_url|         author| bestsellers_date|         description|        price|   published_date|    publisher|rank|rank_last_week|    title|weeks_on_list|
+--------------------+--------------------+---------------+-----------------+--------------------+-------------+-----------------+-------------+----+--------------+---------+-------------+
|{5b4aa4ead3089013...|http://www.amazon...|  Dean R Koontz|{{1211587200000}}|Odd Thomas, who c...|   {null, 27}|{{1212883200000}}|       Bantam| {1}|           {0}|ODD HOURS|          {1}|
|{5b4aa4ead3089013...|http://www.amazon...|Stephenie Meyer|{{1211587200000}}|Aliens have taken...|{25.99, null}|{{1212883200000}}|Little, Brown| {2}|           {1}| THE HOST|          {3}|
+--------------------+--------------------+------------

In [11]:
#Return first row
df.head()

Row(_id=Row($oid='5b4aa4ead3089013507db18b'), amazon_product_url='http://www.amazon.com/Odd-Hours-Dean-Koontz/dp/0553807056?tag=NYTBS-20', author='Dean R Koontz', bestsellers_date=Row($date=Row($numberLong='1211587200000')), description='Odd Thomas, who can communicate with the dead, confronts evil forces in a California coastal town.', price=Row($numberDouble=None, $numberInt='27'), published_date=Row($date=Row($numberLong='1212883200000')), publisher='Bantam', rank=Row($numberInt='1'), rank_last_week=Row($numberInt='0'), title='ODD HOURS', weeks_on_list=Row($numberInt='1'))

In [12]:
#Return first n rows()
df.take(4)

[Row(_id=Row($oid='5b4aa4ead3089013507db18b'), amazon_product_url='http://www.amazon.com/Odd-Hours-Dean-Koontz/dp/0553807056?tag=NYTBS-20', author='Dean R Koontz', bestsellers_date=Row($date=Row($numberLong='1211587200000')), description='Odd Thomas, who can communicate with the dead, confronts evil forces in a California coastal town.', price=Row($numberDouble=None, $numberInt='27'), published_date=Row($date=Row($numberLong='1212883200000')), publisher='Bantam', rank=Row($numberInt='1'), rank_last_week=Row($numberInt='0'), title='ODD HOURS', weeks_on_list=Row($numberInt='1')),
 Row(_id=Row($oid='5b4aa4ead3089013507db18c'), amazon_product_url='http://www.amazon.com/The-Host-Novel-Stephenie-Meyer/dp/0316218502?tag=NYTBS-20', author='Stephenie Meyer', bestsellers_date=Row($date=Row($numberLong='1211587200000')), description='Aliens have taken control of the minds and bodies of most humans, but one woman won’t surrender.', price=Row($numberDouble='25.99', $numberInt=None), published_date=

In [13]:
# Compute summary statistics
print(df.describe().show())

23/04/28 06:19:59 WARN package: Truncated the string representation of a plan since it was too large. This behavior can be adjusted by setting 'spark.sql.debug.maxToStringFields'.


+-------+--------------------+---------------+--------------------+---------+------------------+
|summary|  amazon_product_url|         author|         description|publisher|             title|
+-------+--------------------+---------------+--------------------+---------+------------------+
|  count|               10195|          10195|               10195|    10195|             10195|
|   mean|                null|           null|                null|     null|1877.7142857142858|
| stddev|                null|           null|                null|     null| 370.9760613506458|
|    min|http://www.amazon...|        AJ Finn|                    |      ACE|  10TH ANNIVERSARY|
|    max|https://www.amazo...|various authors|’Tis for the Rebe...|allantine|               ZOO|
+-------+--------------------+---------------+--------------------+---------+------------------+

None


In [14]:
#Fetch column names
df.columns

['_id',
 'amazon_product_url',
 'author',
 'bestsellers_date',
 'description',
 'price',
 'published_date',
 'publisher',
 'rank',
 'rank_last_week',
 'title',
 'weeks_on_list']

In [15]:
#Count the number of rows in the dataframe
df.count()

10195

In [16]:
#Count distinct row count in dataframe
print(df.distinct().count())

10195


In [17]:
# Prints plans including physical and logical
df.explain()

== Physical Plan ==
FileScan json [_id#8,amazon_product_url#9,author#10,bestsellers_date#11,description#12,price#13,published_date#14,publisher#15,rank#16,rank_last_week#17,title#18,weeks_on_list#19] Batched: false, DataFilters: [], Format: JSON, Location: InMemoryFileIndex(1 paths)[file:/kaggle/input/new-york-times-hardcover-fiction-best-sellers/nyt2...., PartitionFilters: [], PushedFilters: [], ReadSchema: struct<_id:struct<$oid:string>,amazon_product_url:string,author:string,bestsellers_date:struct<$d...




# Common Functions 
## Duplicate Values
- Duplicate values in a table can be eliminated by using dropDuplicates() function

In [18]:
df_dropDups = df.drop_duplicates()
df_dropDups.show(5)

+--------------------+--------------------+---------------+-----------------+--------------------+-------------+-----------------+-------------+----+--------------+------------------+-------------+
|                 _id|  amazon_product_url|         author| bestsellers_date|         description|        price|   published_date|    publisher|rank|rank_last_week|             title|weeks_on_list|
+--------------------+--------------------+---------------+-----------------+--------------------+-------------+-----------------+-------------+----+--------------+------------------+-------------+
|{5b4aa4ead3089013...|http://www.amazon...|   Daniel Silva|{{1217030400000}}|Gabriel Allon, an...|{26.95, null}|{{1218326400000}}|       Putnam| {1}|           {0}|THE SECRET SERVANT|          {1}|
|{5b4aa4ead3089013...|http://www.amazon...|     Jane Green|{{1218240000000}}|A woman’s life ch...|    {null, 0}|{{1219536000000}}|       Viking|{18}|           {0}|   THE BEACH HOUSE|          {0}|
|{5b4aa4ea

In [19]:
print(df.count())
print(df_dropDups.count())

10195
10195


# Select Operation 

In [20]:
# Show all enteries in title column
df.select("author").show(10)

+--------------------+
|              author|
+--------------------+
|       Dean R Koontz|
|     Stephenie Meyer|
|        Emily Giffin|
|   Patricia Cornwell|
|     Chuck Palahniuk|
|James Patterson a...|
|       John Sandford|
|       Jimmy Buffett|
|    Elizabeth George|
|      David Baldacci|
+--------------------+
only showing top 10 rows



In [21]:
# Show all enteries in title, author, rank, price columns
df.select("author","title","rank","price").show(10)

+--------------------+--------------------+----+-------------+
|              author|               title|rank|        price|
+--------------------+--------------------+----+-------------+
|       Dean R Koontz|           ODD HOURS| {1}|   {null, 27}|
|     Stephenie Meyer|            THE HOST| {2}|{25.99, null}|
|        Emily Giffin|LOVE THE ONE YOU'...| {3}|{24.95, null}|
|   Patricia Cornwell|           THE FRONT| {4}|{22.95, null}|
|     Chuck Palahniuk|               SNUFF| {5}|{24.95, null}|
|James Patterson a...|SUNDAYS AT TIFFANY’S| {6}|{24.99, null}|
|       John Sandford|        PHANTOM PREY| {7}|{26.95, null}|
|       Jimmy Buffett|          SWINE NOT?| {8}|{21.99, null}|
|    Elizabeth George|     CARELESS IN RED| {9}|{27.95, null}|
|      David Baldacci|     THE WHOLE TRUTH|{10}|{26.99, null}|
+--------------------+--------------------+----+-------------+
only showing top 10 rows



# When opertaion 
### show title and assign 0 or 1 depending on title 


In [22]:
df.select("title", when(df.title != 'ODD HOURS', 1).otherwise(0)).show(10)

+--------------------+-----------------------------------------------------+
|               title|CASE WHEN (NOT (title = ODD HOURS)) THEN 1 ELSE 0 END|
+--------------------+-----------------------------------------------------+
|           ODD HOURS|                                                    0|
|            THE HOST|                                                    1|
|LOVE THE ONE YOU'...|                                                    1|
|           THE FRONT|                                                    1|
|               SNUFF|                                                    1|
|SUNDAYS AT TIFFANY’S|                                                    1|
|        PHANTOM PREY|                                                    1|
|          SWINE NOT?|                                                    1|
|     CARELESS IN RED|                                                    1|
|     THE WHOLE TRUTH|                                                    1|

# 'isin' Operation

In [23]:
# shows rows with specified author if in the given option 
df[df.author.isin("John Sandford", "Emily Giffin")].show()

+--------------------+--------------------+-------------+-----------------+--------------------+-------------+-----------------+------------+----+--------------+--------------------+-------------+
|                 _id|  amazon_product_url|       author| bestsellers_date|         description|        price|   published_date|   publisher|rank|rank_last_week|               title|weeks_on_list|
+--------------------+--------------------+-------------+-----------------+--------------------+-------------+-----------------+------------+----+--------------+--------------------+-------------+
|{5b4aa4ead3089013...|http://www.amazon...| Emily Giffin|{{1211587200000}}|A woman's happy m...|{24.95, null}|{{1212883200000}}|St. Martin's| {3}|           {2}|LOVE THE ONE YOU'...|          {2}|
|{5b4aa4ead3089013...|http://www.amazon...|John Sandford|{{1211587200000}}|The Minneapolis d...|{26.95, null}|{{1212883200000}}|      Putnam| {7}|           {4}|        PHANTOM PREY|          {3}|
|{5b4aa4ead3089

# Like Operation 


In [24]:
# Show author and title is True if title has  "THE" word in titles 
df.select("author","title",df.title.like("% THE %")).show(15)

+--------------------+--------------------+------------------+
|              author|               title|title LIKE % THE %|
+--------------------+--------------------+------------------+
|       Dean R Koontz|           ODD HOURS|             false|
|     Stephenie Meyer|            THE HOST|             false|
|        Emily Giffin|LOVE THE ONE YOU'...|              true|
|   Patricia Cornwell|           THE FRONT|             false|
|     Chuck Palahniuk|               SNUFF|             false|
|James Patterson a...|SUNDAYS AT TIFFANY’S|             false|
|       John Sandford|        PHANTOM PREY|             false|
|       Jimmy Buffett|          SWINE NOT?|             false|
|    Elizabeth George|     CARELESS IN RED|             false|
|      David Baldacci|     THE WHOLE TRUTH|             false|
|        Troy Denning|          INVINCIBLE|             false|
|          James Frey|BRIGHT SHINY MORNING|             false|
|         Garth Stein|THE ART OF RACING...|            

# StartsWith - EndsWith Operation 
- StartsWith scans from beginning of word/content with specified criteria in brackets.
- EndsWith processes the word/content starting from the end.
- Both are case sensitive


In [25]:
df.select("author","title",df.title.startswith("THE")).show(5)
df.select("author","title",df.title.endswith("NT")).show(5)

+-----------------+--------------------+----------------------+
|           author|               title|startswith(title, THE)|
+-----------------+--------------------+----------------------+
|    Dean R Koontz|           ODD HOURS|                 false|
|  Stephenie Meyer|            THE HOST|                  true|
|     Emily Giffin|LOVE THE ONE YOU'...|                 false|
|Patricia Cornwell|           THE FRONT|                  true|
|  Chuck Palahniuk|               SNUFF|                 false|
+-----------------+--------------------+----------------------+
only showing top 5 rows

+-----------------+--------------------+-------------------+
|           author|               title|endswith(title, NT)|
+-----------------+--------------------+-------------------+
|    Dean R Koontz|           ODD HOURS|              false|
|  Stephenie Meyer|            THE HOST|              false|
|     Emily Giffin|LOVE THE ONE YOU'...|              false|
|Patricia Cornwell|           THE

# Substring Operations 
- In the following examples, texts are extracted from the index numbers (1, 3), (2, 3) and (1, 6).

In [26]:
df.select(df.author.substr(1,3).alias("title")).show(3)
df.select(df.author.substr(2,3).alias("title")).show(3)
df.select(df.author.substr(1,6).alias("title")).show(3)

+-----+
|title|
+-----+
|  Dea|
|  Ste|
|  Emi|
+-----+
only showing top 3 rows

+-----+
|title|
+-----+
|  ean|
|  tep|
|  mil|
+-----+
only showing top 3 rows

+------+
| title|
+------+
|Dean R|
|Stephe|
|Emily |
+------+
only showing top 3 rows



# Adding Column 

In [27]:
from pyspark.sql import functions as F
# Lit() is required while we are creating columns with exact values

df = df.withColumn("new_column",F.lit('This_is_a_new_Column'))
display(df)

DataFrame[_id: struct<$oid:string>, amazon_product_url: string, author: string, bestsellers_date: struct<$date:struct<$numberLong:string>>, description: string, price: struct<$numberDouble:string,$numberInt:string>, published_date: struct<$date:struct<$numberLong:string>>, publisher: string, rank: struct<$numberInt:string>, rank_last_week: struct<$numberInt:string>, title: string, weeks_on_list: struct<$numberInt:string>, new_column: string]

# Updating Columns 
#### update column 'amazon_product_url' with 'URL'

In [28]:
df = df.withColumnRenamed("amazon_product_url","URL")
df.show(3)

+--------------------+--------------------+---------------+-----------------+--------------------+-------------+-----------------+-------------+----+--------------+--------------------+-------------+--------------------+
|                 _id|                 URL|         author| bestsellers_date|         description|        price|   published_date|    publisher|rank|rank_last_week|               title|weeks_on_list|          new_column|
+--------------------+--------------------+---------------+-----------------+--------------------+-------------+-----------------+-------------+----+--------------+--------------------+-------------+--------------------+
|{5b4aa4ead3089013...|http://www.amazon...|  Dean R Koontz|{{1211587200000}}|Odd Thomas, who c...|   {null, 27}|{{1212883200000}}|       Bantam| {1}|           {0}|           ODD HOURS|          {1}|This_is_a_new_Column|
|{5b4aa4ead3089013...|http://www.amazon...|Stephenie Meyer|{{1211587200000}}|Aliens have taken...|{25.99, null}|{{12

# Removing Column 
- Removal of a column can be achieved in two ways: \

- Adding the list of column names in the drop() function
- Specifying columns by pointing in the drop function

In [29]:
# 1 
df_remove = df.drop("publisher","published_date").show(5)

# 2 
df_remove2 = df.drop(df.publisher).drop(df.published_date).show(5)


+--------------------+--------------------+-----------------+-----------------+--------------------+-------------+----+--------------+--------------------+-------------+--------------------+
|                 _id|                 URL|           author| bestsellers_date|         description|        price|rank|rank_last_week|               title|weeks_on_list|          new_column|
+--------------------+--------------------+-----------------+-----------------+--------------------+-------------+----+--------------+--------------------+-------------+--------------------+
|{5b4aa4ead3089013...|http://www.amazon...|    Dean R Koontz|{{1211587200000}}|Odd Thomas, who c...|   {null, 27}| {1}|           {0}|           ODD HOURS|          {1}|This_is_a_new_Column|
|{5b4aa4ead3089013...|http://www.amazon...|  Stephenie Meyer|{{1211587200000}}|Aliens have taken...|{25.99, null}| {2}|           {1}|            THE HOST|          {3}|This_is_a_new_Column|
|{5b4aa4ead3089013...|http://www.amazon...|  

# GroupBy Operations

In [30]:
df.groupBy("author").count().show(10)

+-----------------+-----+
|           author|count|
+-----------------+-----+
|       James Frey|    2|
| Elin Hilderbrand|   58|
|Sharon Kay Penman|    2|
|      Lisa Genova|    7|
|     Will Allison|    1|
|Patricia Cornwell|   64|
|    Laurie R King|    6|
|       Tea Obreht|    6|
|     Sarah Dunant|    1|
|     Tim Johnston|    1|
+-----------------+-----+
only showing top 10 rows



# Filter Operation 
- Filtering entries of title
- Only keeps records having value 'THE HOST'

In [31]:
df.filter(df.title == 'THE HOST').show(5)

+--------------------+--------------------+---------------+-----------------+--------------------+-------------+-----------------+-------------+----+--------------+--------+-------------+--------------------+
|                 _id|                 URL|         author| bestsellers_date|         description|        price|   published_date|    publisher|rank|rank_last_week|   title|weeks_on_list|          new_column|
+--------------------+--------------------+---------------+-----------------+--------------------+-------------+-----------------+-------------+----+--------------+--------+-------------+--------------------+
|{5b4aa4ead3089013...|http://www.amazon...|Stephenie Meyer|{{1211587200000}}|Aliens have taken...|{25.99, null}|{{1212883200000}}|Little, Brown| {2}|           {1}|THE HOST|          {3}|This_is_a_new_Column|
|{5b4aa4ead3089013...|http://www.amazon...|Stephenie Meyer|{{1212192000000}}|Aliens have taken...|{25.99, null}|{{1213488000000}}|Little, Brown| {2}|           {2}|

# Handling Missing Values 

## Replacing null values
- dataframe.na.fill()
- dataFrame.fillna()
- dataFrameNaFunctions.fill()

## Returning new dataframe restricting rows with null valuesdataframe.na.drop()
- dataFrame.dropna()
- dataFrameNaFunctions.drop()

## Return new dataframe replacing one value with another
- dataframe.na.replace(5, 15)
- dataFrame.replace()
- dataFrameNaFunctions.replace()


# Repartitioning
- It is possible to increase or decrease the existing level of partitioning in RDD. \

- Increasing can be actualized by using repartition(self, numPartitions) function which results in a new RDD that obtains same/higher number of partitions. \

- Decreasing can be processed with coalesce(self, numPartitions, shuffle=False) function that results in new RDD with a reduced number of partitions to a specified number

In [32]:
# Dataframe with 10 partitions
df.repartition(10).rdd.getNumPartitions()

# Dataframe with 1 partition
df.coalesce(1).rdd.getNumPartitions()

1

# Running SQL commands in Spark 

In [33]:
# Registering a table 
df.registerTempTable("df_temp")

/opt/conda/lib/python3.7/site-packages/pyspark/sql/dataframe.py:330: FutureWarning: Deprecated in 2.0, use createOrReplaceTempView instead.
  warnings.warn("Deprecated in 2.0, use createOrReplaceTempView instead.", FutureWarning)


In [34]:
sc.sql("Select * from df_temp").show(3)

+--------------------+--------------------+---------------+-----------------+--------------------+-------------+-----------------+-------------+----+--------------+--------------------+-------------+--------------------+
|                 _id|                 URL|         author| bestsellers_date|         description|        price|   published_date|    publisher|rank|rank_last_week|               title|weeks_on_list|          new_column|
+--------------------+--------------------+---------------+-----------------+--------------------+-------------+-----------------+-------------+----+--------------+--------------------+-------------+--------------------+
|{5b4aa4ead3089013...|http://www.amazon...|  Dean R Koontz|{{1211587200000}}|Odd Thomas, who c...|   {null, 27}|{{1212883200000}}|       Bantam| {1}|           {0}|           ODD HOURS|          {1}|This_is_a_new_Column|
|{5b4aa4ead3089013...|http://www.amazon...|Stephenie Meyer|{{1211587200000}}|Aliens have taken...|{25.99, null}|{{12

In [35]:
sc.sql("SELECT CASE WHEN description LIKE '%love%' THEN 'Love_Theme' +          
           WHEN description LIKE '%hate%' THEN 'Hate_Theme' +            
           WHEN description LIKE '%happy%' THEN 'Happiness_Theme' +            
           WHEN description LIKE '%anger%' THEN 'Anger_Theme' +            
           WHEN description LIKE '%horror%' THEN 'Horror_Theme' +              
           WHEN description LIKE '%death%' THEN 'Criminal_Theme' +               
           WHEN description LIKE '%detective%' THEN 'Mystery_Theme' ELSE 'Other_Themes' END Themes from df_temp").groupBy('Themes').count().show()

SyntaxError: EOL while scanning string literal (3770871487.py, line 1)

In [ ]:
sc.sql("SELECT CASE WHEN description LIKE '%love%' THEN 'Love_Theme'             
           WHEN description LIKE '%hate%' THEN 'Hate_Theme'            
           WHEN description LIKE '%happy%' THEN 'Happiness_Theme' \            
           WHEN description LIKE '%anger%' THEN 'Anger_Theme' \             
           WHEN description LIKE '%horror%' THEN 'Horror_Theme' \              
           WHEN description LIKE '%death%' THEN 'Criminal_Theme' \               
           WHEN description LIKE '%detective%' THEN 'Mystery_Theme' ELSE 'Other_Themes' END Themes from df").groupBy('Themes').count().show()